In [1]:
# REAL NOTEBOOK

# This notebook runs a simulation for the Solar Dynamo model with the sABC algorithm.
# The results of the simulation are all stored in files in the directory Simulations/Real i.
# The reason behind this is to have an easier access to the results of already run simulations,
# without running them again. This notebook only processes real data!!

# NB: In this notebook, there's no function to visualize the results: the visualization is all
# contained in the "visualization_real.ipynb" notebook.

# RULES:

# There are two ways to use this notebook:
# 1) change all the parameters and then run all -> correct way
# 2) change things randomly and not in order and then run -> wrong way
# Please be careful, some functions change the directory in which everything is being saved; the
# order of the calling of the functions is organized to start from a generic dir, create the dir
# Simulations/Simulation i, go to that directory and then, when everythingis finished, go back to
# the initial dir!!! If you don't do that, it will stay in the subdir and at the next run it will
# create a subdir of a subdir -> if you need to stop midway through because u forgot something, 
# remember to come back to the initial directory (and eliminate the directory that has not correct
# files inside).

# GG EZ - kallo27

In [2]:
# REQUIRED PACKAGES -> no visualization!!

using SpecialFunctions
using Distributions
using SimulatedAnnealingABC
using DataFrames
using XLSX
using ThreadPinning

In [3]:
# MODEL AND MANAGING FUNCTIONS FILES

push!(LOAD_PATH, "./src")
include("../src/DirUtils.jl")
include("../src/SDDESolarDynamo.jl")

using .DirUtils
using .SDDESolarDynamo

In [4]:
# THREADS SETTINGS AND INFO

ThreadPinning.pinthreads(:cores)
ThreadPinning.threadinfo()

Hostname: 	DESKTOP-7P7E403
CPU(s): 	1 x AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx
CPU target: 	znver1
Cores: 		4 (8 CPU-threads due to 2-way SMT)
NUMA domains: 	1 (4 cores each)

Julia threads: 	4

CPU socket 1
  0,1, 2,3, 4,5, 6,7


# = Julia thread, # = Julia thread on HT, # = >1 Julia thread

(Mapping: 1 => 0, 2 => 2, 3 => 4, 4 => 6,)


In [10]:
# EXTRACTING OPEN MAGNETIC FLUX AND SUNSPOT NUMBER RECORDS FROM XLSX FILE

# Define DataFrame object
data = load_sunspots_data("SN Usoskin Brehm.xlsx")

# Creation of the summary statistics for the real data
u = data.open_magn_flux.^2
indices = [39, 43, 49, 55]
sim_ss = reduced_fourier_spectrum(u, indices)

4-element Vector{Float64}:
  466.58123779466064
 1324.3581906466002
  763.6975827651291
 1361.8222861793029

In [11]:
# DIRECTORY MANAGING

# Current directory
initial_directory = pwd()

# New directory
create_directory("real")

# NB: After "create_directory", we move to the new directory.
# DON'T RUN THIS AGAIN, wait for the simulation to finish!!!! If you made errors,
# eliminate the Simulations/Simulation i directory and then rerun everything

Directory created at: /mnt/c/Users/Utente/SDDESolarDynamo.jl/examples/real_data_sim/real2


In [12]:
# SIMULATION PARAMETERS MANAGING

# Parameters that can be tuned for new simulations
prior = product_distribution(Uniform(5.0, 15), Uniform(0.1, 10.0), Uniform(0.1, 6.0), Uniform(0.01, 0.3), Uniform(1, 15))
n_particles = 1000
n_simulation = 50000
v = 1.0
type = 1

# Writing on file "sabc_params.csv" of the values set in this cell.
save_sabc_params(prior, n_particles, n_simulation, v, type, indices)

Parameters saved to: /mnt/c/Users/Utente/SDDESolarDynamo.jl/examples/real_data_sim/real2/sabc_params.csv


In [13]:
# SIMULATION

# Conditions
tmin = data.year[1]; tmax = data.year[end]
Tsim = [tmin, tmax]
dt = 0.1

# Actual usage of the sABC algorithm
result = sabc(f_dist, prior;
              n_particles = n_particles, 
              n_simulation = n_simulation,
              v = v,
              type = type,
              indices = indices,
              fourier_data = sim_ss,
              Tsim = Tsim,
              dt = dt)

# Display of the summary of the results
display(result)

# Saving the results to the files: "eps_hist.csv", "u_hist.csv", "rho_hist.csv", "pop.csv", "rho.csv".
save_result(result)

┌ Info: Initialization for 'single_eps'
└ @ SimulatedAnnealingABC /home/kallo27/.julia/packages/SimulatedAnnealingABC/nelUw/src/SimulatedAnnealingABC.jl:158
┌ Info: All particles have been updated 49 times.
└ @ SimulatedAnnealingABC /home/kallo27/.julia/packages/SimulatedAnnealingABC/nelUw/src/SimulatedAnnealingABC.jl:399


Approximate posterior sample with 1000 particles:
  - algorithm: :single_eps
  - simulations used: 50000
  - number of population updates: 49
  - average transformed distance: 0.1484
  - ϵ: [0.06737]
  - number of population resamplings: 1
  - acceptance rate: 0.06067
The sample can be accessed with the field `population`.
The history of ϵ can be accessed with the field `state.ϵ_history`.
The history of ρ can be accessed with the field `state.ρ_history`.
The history of u can be accessed with the field `state.u_history`.


eps_hist.csv data saved to: /mnt/c/Users/Utente/SDDESolarDynamo.jl/examples/real_data_sim/real2/eps_hist.csv
u_hist.csv data saved to: /mnt/c/Users/Utente/SDDESolarDynamo.jl/examples/real_data_sim/real2/u_hist.csv
rho_hist.csv data saved to: /mnt/c/Users/Utente/SDDESolarDynamo.jl/examples/real_data_sim/real2/rho_hist.csv
Posterior parameters saved to: /mnt/c/Users/Utente/SDDESolarDynamo.jl/examples/real_data_sim/real2/pop.csv
Rho values saved to: /mnt/c/Users/Utente/SDDESolarDynamo.jl/examples/real_data_sim/real2/rho.csv


In [14]:
# DIRECTORY MANAGING

# We go back to the initial directory
cd(initial_directory)
pwd()

"/mnt/c/Users/Utente/SDDESolarDynamo.jl/examples"